In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

ImportError: cannot import name 'load_dotenv' from 'dotenv' (/workspaces/curly-barnacle/.venv/lib/python3.10/site-packages/dotenv/__init__.py)